# Week_2 Assignment_1: Выбор числа соседей

https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/eblyU/vybor-chisla-sosiediei

Данное задание основано на материалах лекций по метрическим методам и посвящено подбору числа соседей в методе kNN.

Вы научитесь:
- работать с методом k ближайших соседей
- выбирать в нем параметр k
- правильно готовить данные к использованию в методе kNN

#### Введение

Метрические методы основаны на гипотезе компактности, суть которой состоит в том, что объекты с похожими признаковыми описаниями имеют похожие значения целевой переменной. Если эта гипотеза верна, то строить прогноз для нового объекта можно на основе близких к нему объектов из обучающей выборки — например, путем усреднения их ответов (для регрессии) или путем выбора наиболее популярного среди них класса (для классификации). Методы такого типа и называются метрическими. Они имеют несколько особенностей:

- Процедура обучения, по сути, отсутствует — достаточно лишь запомнить все объекты обучающей выборки
- Можно использовать метрику, учитывающую особенности конкретного набора данных — например, наличие категориальных (номинальных) признаков
- При правильном выборе метрики и достаточном размере обучающей выборки метрические алгоритмы показывают качество, близкое к оптимальному

Метрические методы чувствительны к масштабу признаков — так, если масштаб одного из признаков существенно превосходит масштабы остальных признаков, то их значения практически не будут влиять на ответы алгоритма. Поэтому важно производить масштабирование признаков. Обычно это делается путем вычитания среднего значения признака и деления на стандартное отклонение.

#### Реализация в Scikit-Learn

Метод k ближайших соседей реализован в классе [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). Основным параметром является n_neighbors, который задает число соседей для построения прогноза.

Вам понадобится производить кросс-валидацию по блокам. Кросс-валидация заключается в разделении выборки на m непересекающихся блоков примерно одинакового размера, после чего выполняется m шагов. На i-м шаге i-й блок выступает в качестве тестовой выборки, объединение всех остальных блоков — в качестве обучающей выборки. Соответственно, на каждом шаге алгоритм обучается на некоторой обучающей выборке, после чего вычисляется его качество на тестовой выборке. После выполнения m шагов мы получаем m показателей качества, усреднение которых и дает оценку кросс-валидации. Подробнее вы можете послушать про кросс-валидацию в видео "Проблема переобучения. Методология решения задач машинного обучения" из первого модуля, а также почитать на [Википедии](https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D0%B5%D0%BA%D1%80%D1%91%D1%81%D1%82%D0%BD%D0%B0%D1%8F_%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0)
или в документации [scikit-learn](http://scikit-learn.org/stable/modules/cross_validation.html).

#### Технически кросс-валидация проводится в два этапа:

1. Создается генератор разбиений [sklearn.model_selection.KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold), который задает набор разбиений на обучение и валидацию. Число блоков в кросс-валидации определяется параметром **n_splits**. Обратите внимание, что порядок следования объектов в выборке может быть неслучайным, это может привести к смещенности кросс-валидационной оценки. Чтобы устранить такой эффект, объекты выборки случайно перемешивают перед разбиением на блоки. Для перемешивания достаточно передать генератору KFold параметр **shuffle=True**.

2. Вычислить качество на всех разбиениях можно при помощи функции [sklearn.model_selection.cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html?highlight=cross_val_score#sklearn.model_selection.cross_val_score). В качестве параметра **estimator** передается классификатор, в качестве параметра **cv** — генератор разбиений с предыдущего шага. С помощью параметра **scoring** можно задавать меру качества, по умолчанию в задачах классификации используется доля верных ответов (accuracy). Результатом является массив, значения которого нужно усреднить. 

Приведение признаков к одному масштабу можно делать с помощью функции sklearn.preprocessing.scale, которой на вход необходимо подать матрицу признаков и получить масштабированную матрицу, в которой каждый столбец имеет нулевое среднее значение и единичное стандартное отклонение.

 
#### Инструкция по выполнению

В этом задании вам нужно подобрать оптимальное значение k для алгоритма kNN. Будем использовать набор данных Wine, где требуется предсказать сорт винограда, из которого изготовлено вино, используя результаты химических анализов.
https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

Выполните следующие шаги:

1. Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data (файл также приложен к этому заданию)
2. Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names, приложенный к заданию)
3. Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).
4. Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.
5. Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. Снова найдите оптимальное k на кросс-валидации.
6. Какое значение k получилось оптимальным после приведения признаков к одному масштабу? Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?

Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.5. При необходимости округляйте дробную часть до двух знаков.

## Ответы на вопросы задания

In [1]:
# load libraries
import pandas as pd
import numpy as np
# import ...

### 1. Загрузите выборку Wine

In [2]:
# load data from file
data = pd.read_csv('wine.data', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
data.shape

(178, 14)

In [4]:
data.dtypes

0       int64
1     float64
2     float64
3     float64
4     float64
5       int64
6     float64
7     float64
8     float64
9     float64
10    float64
11    float64
12    float64
13      int64
dtype: object

### 2. Извлеките из данных признаки и классы

Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names, приложенный к заданию)

In [5]:
X = data.values[:,1:]
X

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [6]:
y = data.values[:,0]
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3.])

### 3. Произведите оценку качества

Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).

In [7]:
from sklearn.model_selection import KFold             # загружаем генератор разбиений
kf = KFold(n_splits=5, shuffle=True, random_state=42) # вызываем генератор с заданными параметрами

from sklearn.neighbors import KNeighborsClassifier    # загружаем классификатор KNN
nn = 3                                                # потому что y имеет три значения
knnc = KNeighborsClassifier(n_neighbors=nn)           # вызываем классификатр с заданными параметрами

In [8]:
# способ первый: вручную через цикл
acc_1 = np.array([])                                      # создаем пустой массив точностей
for train_index, test_index in kf.split(X):               # запускаем цикл по всем наборам разбиений
    X_train, X_test = X[train_index], X[test_index]       # определяем очередной обучающий набор
    y_train, y_test = y[train_index], y[test_index]       # определяем очередной проверочный набор
    a = knnc.fit(X_train, y_train).score(X_test, y_test)  # обучаем классификатор, получаем точность
    acc_1 = np.append(acc_1, a)                           # добавляем значение точности в массив точностей
acc_1

array([0.80555556, 0.61111111, 0.63888889, 0.65714286, 0.82857143])

In [9]:
# среднее от массива значений точностей
acc_1_mean = np.mean(acc_1)
acc_1_mean

0.7082539682539682

In [10]:
# способ второй: с помощью cross_val_score
from sklearn.model_selection import cross_val_score          # импортируем функцию оценки качества модели
acc_2 = cross_val_score(knnc, X, y, cv=kf)
acc_2

array([0.80555556, 0.61111111, 0.63888889, 0.65714286, 0.82857143])

In [11]:
# среднее от массива значений точностей
acc_2_mean = np.mean(acc_2)
acc_2_mean

0.7082539682539682

получилось такое же значение acc_2_mean, как и при "ручном" методе. Дальше будем испльзовать cross_val_score.

### 4. Найдите точность классификации на кросс-валидации для метода k ближайших соседей при k от 1 до 50

При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2

In [12]:
# пустая таблица для складирования результатов цикла
df_ = pd.DataFrame()

# цикл с использованием метода cross_val_score
for nn in range (1, 51):
    knnc = KNeighborsClassifier(n_neighbors=nn)  
    acc = cross_val_score(knnc, X, y, cv=kf)
    acc_mean = np.mean(acc)
    df_ = df_.append({'neighbours':nn, 'accuracy':acc_mean}, ignore_index=True)

df_.sort_values(by=['accuracy'], ascending=False).head(5)

,accuracy,neighbours
0,0.730476,1.0
33,0.724603,34.0
34,0.724603,35.0
47,0.719048,48.0
35,0.713492,36.0


Удивительно, но наибольшая точность достигается при nn=0 то есть KNN, по сути, не задействован.

### 5. Произведите масштабирование признаков

с помощью функции [sklearn.preprocessing.scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html?highlight=sklearn%20preprocessing%20scale#sklearn.preprocessing.scale). Снова найдите оптимальное k на кросс-валидации.

In [14]:
from sklearn import preprocessing
X = preprocessing.scale(X)

In [17]:
# цикл с использованием метода cross_val_score
df_ = pd.DataFrame()

for nn in range (1, 51):
    knnc = KNeighborsClassifier(n_neighbors=nn)  
    acc = cross_val_score(knnc, X, y, cv=kf)
    acc_mean = np.mean(acc)
    df_ = df_.append({'neighbours':nn, 'accuracy':acc_mean}, ignore_index=True)

df_.sort_values(by=['accuracy'], ascending=False).head(5)

,accuracy,neighbours
28,0.977619,29.0
15,0.972063,16.0
14,0.972063,15.0
16,0.966508,17.0
19,0.966508,20.0


Видно, что:
- после масштабирования точность метода драматически выросла (с 73.0% до 97.8%)
- лучшую точность дает KNN при 29 соседях. Что выглядит разумнее, чем 0 соседей без масштабирования.

In [18]:
# вытащим значение точности при nn=3 (как в вопросе №3)
acc_3 = df_.iloc[2]
acc_3

accuracy      0.955079
neighbours    3.000000
Name: 2, dtype: float64

При, казалось бы, напрашивающемся числе соседей nn=3 точность существенно ниже (acc = 95.5%).

### 6. Какое значение k получилось оптимальным после приведения признаков к одному масштабу?

Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?

**Ответ: 29**